In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,time_idx,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,i64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,0,"""a""",0.188315,0.141979,0,10000,-10000,0,0.710738,"""a"""
0.841471,1,"""a""",0.622668,0.866021,1,10001,-9999,1,0.133336,"""a"""
0.909297,2,"""a""",0.415737,0.771208,2,10002,-9998,1,0.542912,"""a"""
0.14112,3,"""a""",0.898933,0.737424,3,10003,-9997,1,0.457373,"""a"""
-0.756802,4,"""a""",0.002624,0.370457,4,10004,-9996,0,0.517487,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.query_jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Convolution (by FFT). 
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full"),
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full"),
).head()

f,a,b
f64,f64,f64
0.0,-0.188315,-0.141979
-0.841471,-0.622668,-0.866021
-0.909297,-0.415737,-0.771208
-0.14112,-0.898933,-0.737424
0.756802,0.185691,-0.228478


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[2.0, -1.0]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # str | pl.Expr
        "x1", "x2",
        target = pl.col("y"),
        add_bias=False
    ).alias("report")
).unnest("report")

idx,coeff,std_err,t,p>|t|
u16,f64,f64,f64,f64
0,2.0,2.3854e-16,8.3842e15,0.0
1,-1.0,9.0158e-17,-1.1092e16,0.0


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

y
list[f64]
"[2.0, -1.0]"


In [10]:
df.select(
    "dummy",
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
).head() 

dummy,coeffs
str,list[f64]
"""a""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.query_lstsq(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
i64,i64,i64,f64,f64
0,10000,-10000,-10000.0,5.8208e-11
1,10001,-9999,-9999.0,5.8208e-11
2,10002,-9998,-9998.0,5.8208e-11
3,10003,-9997,-9997.0,5.8208e-11
4,10004,-9996,-9996.0,5.8208e-11


In [12]:
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"


In [13]:
# Rolling regression, kind of slow rn
df.lazy().rolling(
    index_column = pl.col("time_idx").set_sorted(),
    period = "30i",
    # offset = "-1i"
).agg(
    pds.query_lstsq(pl.col("x1"), pl.col("x2"), target = pl.col("y"), add_bias=False).alias("coefficients")
).slice(offset = 30).select(
    "time_idx",
    "coefficients",
).collect().head(10)

time_idx,coefficients
i64,list[f64]
30,"[2.0, -1.0]"
31,"[2.0, -1.0]"
32,"[2.0, -1.0]"
33,"[2.0, -1.0]"
34,"[2.0, -1.0]"
35,"[2.0, -1.0]"
36,"[2.0, -1.0]"
37,"[2.0, -1.0]"
38,"[2.0, -1.0]"


In [14]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [15]:
# Only want singular values (principal values?)
df.select(
    pds.query_singular_values("a", "b", "x1")
)

a
list[f64]
"[288675.133152, 29.02902, 28.725808]"


In [16]:
# Singular values + The principal components
df.select(
    pds.query_pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
29.029919,"[0.382795, 0.923833]"
28.726036,"[0.923833, -0.382795]"


# ML Metrics

In [17]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""a""",0.327183,0.981941,0.530806,0.51161,0.260516,0.529158,0.507766
"""b""",0.33472,1.001556,0.494559,0.491193,0.246435,0.503604,0.497691


# Str Extension

In [18]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [19]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""world"""
"""hello"""
"""to"""
"""church"""
"""going"""


In [20]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
""""""
"""go"""
"""world"""
"""church"""
"""hello"""


In [21]:
df2.select(
    pds.str_leven("word", "world")
).head()

word
u32
2
1
2
1
2


In [22]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", "world")
).head()

word
u32
2
1
2
1
2


In [23]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", "world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [24]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), "world", 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [25]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [26]:
df.filter(
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [27]:
df.select(
    pds.str_leven("word", "asasasa", return_sim=True).alias("asasasa"),
    pds.str_leven("word", "sasaaasss", return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", "asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", "apples").alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", "apples", return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", "apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [28]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
1.277465
0.773817
1.756581


In [29]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.rand_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
).head()

a,random
f64,f64
null,null
null,null
1.277465,-1.674733
0.773817,0.560937
1.756581,0.992039


In [30]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
1.277465,"""3c"""
0.773817,"""AEPS"""
1.756581,"""hDJle"""


In [31]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
1.277465,"""aztcM"""
0.773817,"""1oP4M"""
1.756581,"""ffVUG"""


In [32]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.rand_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.rand_uniform(low = 0., high = None, respect_null=True).alias("test2")
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,test2,test1_perturbed
f64,f64,f64,f64
null,0.340746,null,0.340851
null,-0.658844,null,-0.658872
1.277465,-0.010592,1.452465,-0.010687
0.773817,1.043324,0.138111,1.042867
1.756581,0.928742,0.108943,0.928807


In [33]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.894017,-0.512243,6
null,0.129167,1.514566,7
1.277465,0.501964,-0.672327,2
0.773817,0.427419,-0.974297,4
1.756581,0.236411,1.490783,8


In [34]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
0.047184,0.962373,4.976853,0.083041


In [35]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.580271,0.900833,0,7
1,0.057428,0.845627,4,6
2,0.235893,0.190847,0,5
0,0.406569,0.844753,4,1
1,0.414208,0.182132,1,9


In [36]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-2.172877,0.029813}","{24.271803,0.931677}","{0.766676,0.54678}"


In [37]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{-0.324077,0.7459}","{32.043972,0.657289}","{1.788706,0.128483}"
1,"{-1.287503,0.198008}","{46.973824,0.104186}","{0.56423,0.688655}"
2,"{-2.152551,0.031426}","{47.50072,0.095116}","{0.900611,0.462732}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [38]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [39]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.884202,0.40179,0.526733,0.128066,9.671189,24
1,0.82619,0.753471,0.172811,0.507349,1.421607,18
2,0.529508,0.366602,0.876003,0.712054,0.731871,14
3,0.710466,0.556221,0.822236,0.462236,1.151097,20
4,0.977441,0.541636,0.418693,0.099895,5.021734,12


In [40]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.884202,0.40179,0.526733,0.128066,9.671189,10
1,0.82619,0.753471,0.172811,0.507349,1.421607,227
2,0.529508,0.366602,0.876003,0.712054,0.731871,634
3,0.710466,0.556221,0.822236,0.462236,1.151097,227
4,0.977441,0.541636,0.418693,0.099895,5.021734,8


In [41]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.884202,0.40179,0.526733,0.128066,9.671189,"[0, 1171, … 644]"
1,0.82619,0.753471,0.172811,0.507349,1.421607,"[1, 510, … 903]"
2,0.529508,0.366602,0.876003,0.712054,0.731871,"[2, 1497, … 1277]"
3,0.710466,0.556221,0.822236,0.462236,1.151097,"[3, 851, … 1478]"
4,0.977441,0.541636,0.418693,0.099895,5.021734,"[4, 1660, … 1618]"


In [42]:
# Get all neighbors within radius r
# The point itself is always considered a neighbor to itself.
print(df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head())

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 1171, … 72]   ┆ 194                │
│ 1   ┆ [1, 510, … 1519]  ┆ 195                │
│ 2   ┆ [2, 1497, … 1318] ┆ 212                │
│ 3   ┆ [3, 851, … 129]   ┆ 230                │
│ 4   ┆ [4, 1660, … 510]  ┆ 148                │
└─────┴───────────────────┴────────────────────┘


In [43]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.884202,0.40179,0.526733,0.128066,9.671189,"[0, 1171, … 644]","[0.0, 0.000366, … 0.003606]"
1,0.82619,0.753471,0.172811,0.507349,1.421607,"[1, 510, … 903]","[0.0, 0.002871, … 0.004004]"
2,0.529508,0.366602,0.876003,0.712054,0.731871,"[2, 1497, … 1277]","[0.0, 0.002804, … 0.005115]"
3,0.710466,0.556221,0.822236,0.462236,1.151097,"[3, 851, … 1478]","[0.0, 0.000489, … 0.002082]"
4,0.977441,0.541636,0.418693,0.099895,5.021734,"[4, 1660, … 1618]","[0.0, 0.000305, … 0.001624]"


In [44]:
# Filter to only points near the given point
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.884202,0.40179,0.526733,0.128066,9.671189
2,0.529508,0.366602,0.876003,0.712054,0.731871
3,0.710466,0.556221,0.822236,0.462236,1.151097
5,0.547667,0.10265,0.448592,0.923002,4.684824
6,0.202616,0.348574,0.472191,0.6189,2.650805


In [45]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
22,0.519091,0.429467,0.50597,0.192174,3.436689
33,0.534265,0.492054,0.22822,0.026486,3.092788
106,0.457548,0.525063,0.565281,0.319523,7.2418
248,0.439614,0.49293,0.277917,0.86758,4.339844
326,0.430246,0.554727,0.099945,0.333985,4.437238


In [46]:
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
106,0.457548,0.525063,0.565281,0.319523,7.2418
351,0.495389,0.469025,0.560574,0.904583,6.3331
481,0.559363,0.478895,0.479673,0.757188,7.459941
603,0.497273,0.510896,0.866498,0.561535,1.539249
790,0.516794,0.536396,0.540608,0.694374,8.291199


In [47]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1833, … 1403]",111
1,"[1, 1268, … 1192]",125
2,"[2, 1416, … 1413]",128
3,"[3, 1674, … 169]",130
4,"[4, 275, … 1962]",74


# Simple Graph Queries

There is limited functionality in the Graph module currently. E.g. Only constant cost per edge.

Graph queries are very expensive.

In [48]:
# friends.select(
#     pl.col("friends").graph.eigen_centrality() # .arg_max()
# ).head()

In [49]:
# Turn friends to a table suitable for graph analytics
df_graph = friends.select(
    pl.col("id"),
    pl.col("friends"),
).explode(pl.col("friends")).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("friends").cast(pl.UInt32),
)
df_graph.head()

id,friends
u32,u32
0,0
0,1833
0,1171
0,1265
0,1549


In [50]:
df_graph.select(
    # Shortest path to the node with id = 3
    # Node and link can be str | pl.Expr
    pds.query_shortest_path(node = "id", link = pl.col("friends"), target = 3, cost = None, parallel=True).alias("shortest_path")
).unnest("shortest_path").sort("id")

id,path
u32,list[u32]
0,"[546, 3]"
1,"[372, 3]"
2,"[1961, 3]"
3,[]
4,"[1625, 3]"
…,…
1995,"[203, 811, … 3]"
1996,"[769, 1648, … 3]"
1997,"[1929, 1026, … 3]"


In [51]:
df_graph.select(
    # Almost every node can reach node 3, and the number is the number steps to reach it
    # This is a way faster way to filter results if you don't need the actual path
    pds.query_node_reachable("id", "friends", target = 3).alias("reach")
).unnest("reach")

id,reachable,steps
u32,bool,u32
1908,true,2
1502,true,4
913,true,5
1255,true,5
1381,true,4
…,…,…
520,true,2
1831,true,4
1281,true,5


In [52]:
relationships = pl.DataFrame({
    "id": range(5),
    "connections":[[1,2,3,4], [2,3], [4], [0,1,2], [1]],
    # Small values means closer
    "close-ness":[[0.4, 0.3, 0.2, 0.1], [0.1, 1.0], [0.5], [0.1, 0.1, 0.1], [0.1]]
}).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("connections").list.eval(pl.element().cast(pl.UInt32))
).explode(
    pl.col("connections"), pl.col("close-ness")
)

relationships.head(50)

id,connections,close-ness
u32,u32,f64
0,1,0.4
0,2,0.3
0,3,0.2
0,4,0.1
1,2,0.1
…,…,…
2,4,0.5
3,0,0.1
3,1,0.1


In [53]:
# To go to node at id = 1, node 0 would rather go to 4 first and then 1.
relationships.select(
    pds.query_shortest_path("id", "connections", target = 1, cost = "close-ness").alias("path")
).unnest("path").head()

id,path,cost
u32,list[u32],f64
1,[],0.0
4,[1],0.1
2,"[4, 1]",0.6
0,"[4, 1]",0.2
3,[1],0.1


In [54]:
# In and out deg
relationships.select(
    pds.query_node_deg("id", "connections", directed=True).alias("deg")
).unnest("deg")

node,deg
u32,u32
1,2
4,1
2,1
0,4
3,3


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [55]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [56]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 1, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [57]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [58]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        threshold = 4,
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [59]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pds.query_similar_words(
        "a",
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"
